https://hackernoon.com/merkle-tree-introduction-4c44250e2da7

What you have to implement
* Treehash algorithm and classical traversal algorithms
    * Leaves
    * Auths
    * stacks

## Authentication
* Output leaf value together with authentication path
* Store left most nodes: $stack_i = a_{i 0}$
* Store right brothers of left most nodes: $auth_i = a_{i 1}$
* Needs an update if $leaf + 1$ is a multiple of $2^h$
    * $auth_h = POP(stck_h)$
    * After $2^h$, needs an update again

In [158]:
import os
import math
import hashlib
from copy import deepcopy

import numpy as np


class Node(object):
    def __init__(self, hash_val, left=None, right=None):
        self.hash = hash_val
        self.left = left
        self.right = right
        if left is None:
            self.height = 0
        else:
            self.height = self.left.height + 1
        self.parent = None
        
    @property
    def sibling(self):
        if self.is_left:
            return self.parent.right
        else:
            return self.parent.left
        
    @property
    def is_left(self):
        return self.parent.left.hash == self.hash


class MarkelTree(object):
    def __init__(self, items):
        self.leaves = list(map(self._make_leaf, map(self._hash, items)))
        self.node_table = dict()
        self.stack = list()
        self.max_height = int(np.log2(len(items)))
        self.is_built = False
        # Initialization
        node = self.leaves.pop()
        self.stack.append(node)
        self.build_tree()

    def _make_leaf(self, hash_val):
        return Node(hash_val)
    
    def _make_parent(self, left, right):
        par_hash = self._hash(left.hash + right.hash)
        parent = Node(par_hash, left, right)
        left.parent = parent
        right.parent = parent
        return parent
    
    def _hash(self, x):
        hash_func = hashlib.sha256()
        hash_func.update(str(x).encode('utf-8'))
        return hash_func.digest()
    
    def build_tree(self):
        while self.stack[-1].height < self.max_height:
            if len(self.stack) >= 2\
                    and self.stack[-1].height == self.stack[-2].height:
                right = self.stack.pop()
                left = self.stack.pop()
                par_node = self._make_parent(left, right)
                self.node_table[par_node.hash] = par_node
                self.stack.append(par_node)
            else:
                new_leaf = self.leaves.pop()
                self.node_table[new_leaf.hash] = new_leaf
                self.stack.append(new_leaf)
        # Store root information
        self.root_node = self.stack[-1]
        self.root_hash = self.root_node.hash
        self.is_built = True
    
    
    def get_authpath(self, item):
        """ Returns an authentication path for an item (not hashed) in 
            the Merkle tree as a list in order from the top of the tree
            to the bottom.
        """
        if not self.is_built:
            raise Exception("The Merkle Tree must be built before an \
                    authentication path is found.")

        hash_val = self._hash(item)

        if not hash_val in self.leaf_hashes:
            raise Exception("The requested item is not in the merkle tree.")

        return self._get_authpath_by_hash(hash_val)
    
    def _get_authpath_by_hash(self, hash_val):
        """ Returns an authentication path as a list in order from the top
            to the bottom of the tree (assumes preconditions have been checked).
        """
        path = []
        while hash_val != self.root_hash:
            node = self.node_table[hash_val]
            sibling = node.sibling
            path.append(sibling)
            hash_val = node.parent.hash
        return path
    
    
    def authenticate(self, data):
        path = self.get_authpath(data)
        _hash = self._hash(data)
        for node in path:
            if node.is_left:
                _hash = self._hash(node.hash + _hash)
            else:
                _hash = self._hash(_hash + node.hash)
        return _hash == self.root_hash
        
        
    @property
    def leaf_hashes(self):
        return [hash_val for hash_val, node in self.node_table.items() if node.height == 0]

In [159]:
n = 4
items = np.arange(2 ** n)
tree = MarkelTree(items)
tree.authenticate(6)

True

In [118]:
path = tree.get_authpath(3)

In [122]:
[x.height for x in path]

[2, 2, 1, 0]

In [121]:
path[0].hight

AttributeError: 'Node' object has no attribute 'hight'

In [112]:
x.reverse()

In [113]:
x

[2, 1, 0]